In [1]:
%load_ext autoreload
%autoreload 2

# Get embeddings of sentences

Get questions in train.json

In [10]:
from utils import *
import os

dataset_root = '../MATH/train/'

with open(os.path.join(dataset_root, 'train.json'), 'r') as f:
    data = json.load(f)
print(len(data))
# print(data[1]['1']['conversations'])
print(data[1])

7500
{'1': {'conversations': [{'from': 'human', 'value': 'What is the value of $-a-b^3+ab$ if $a=-3$ and $b=2$?'}, {'from': 'gpt', 'value': 'Plugging in the given values yields $-a-b^3+ab=-(-3)-2^3+(-3)(2)=3-8-6=\\boxed{-11}$.'}], 'system': 'You are a helpful assistant skilled in solving mathematical problems. Provide step-by-step explanations for each solution and ensure clarity in reasoning.', 'tools': {'calculator': {'description': 'A basic calculator tool to perform arithmetic operations.', 'enabled': True, 'functions': ['add', 'subtract', 'multiply', 'divide']}, 'algebra_solver': {'description': 'An algebra solver tool to simplify equations and solve for unknowns.', 'enabled': True, 'functions': ['simplify', 'solve']}}}}


In [3]:
# Extract human questions
questions = []
for item in data:
    for key, conversations in item.items():
        for conv in conversations['conversations']:
            if conv['from'] == 'human':
                questions.append(conv['value'])

# Print the total number of questions and the first few
print(f"Total number of questions: {len(questions)}")
print("\nFirst 5 questions:")
for q in questions[:5]:
    print(q)

Total number of questions: 7500

First 5 questions:
A rectangle having integer length and width has a perimeter of 100 units. What is the number of square units in the least possible area?
What is the value of $-a-b^3+ab$ if $a=-3$ and $b=2$?
Find the radius of the circle with equation $x^2 - 6x + y^2 + 2y + 6 = 0$.
Solve for $x$: $4x^{1/3}-2 \cdot \frac{x}{x^{2/3}}=7+\sqrt[3]{x}$.
What is the sum of all of the solutions of the equation $\frac{4x}{20}=\frac{5}{x}$?


In [4]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

# Load a lightweight transformer model
model = SentenceTransformer('all-distilroberta-v1', device=device)

# Compute sentence embeddings
embeddings = model.encode(questions, convert_to_tensor=True)

# Optional: check tensor shape
print("Embedding shape:", embeddings.shape)

Device: cuda
Embedding shape: torch.Size([7500, 768])


In [5]:
group_numbers = cluster_embeddings(embeddings, num_clusters=20)  # Example: 10 clusters

# Print the first 10 group numbers
print(group_numbers[:10])

[13, 15, 8, 15, 15, 8, 8, 10, 15, 17]


In [6]:
print(count_items_in_groups(group_numbers))

{13: 189, 15: 583, 8: 409, 10: 301, 17: 412, 2: 329, 5: 412, 14: 316, 12: 699, 3: 374, 0: 360, 7: 297, 18: 514, 19: 272, 4: 245, 6: 377, 1: 418, 9: 355, 11: 242, 16: 396}


Choose the top 8 question that is most close to the mean of each group.

In [7]:
import numpy as np

def select_top_questions_per_group(group_numbers, questions, embeddings):
    # Convert inputs to numpy arrays for efficient processing
    group_numbers = np.array(group_numbers)
    embeddings = np.array(embeddings.cpu())
    
    # Get unique groups
    unique_groups = np.unique(group_numbers)
    
    # Dictionary to store top 8 indices for each group
    top_questions_per_group = {}
    
    for group in unique_groups:
        # Find indices of questions in this group
        group_indices = np.where(group_numbers == group)[0]
        
        # Get embeddings for this group
        group_embeddings = embeddings[group_indices]
        
        # Calculate the mean embedding for the group
        group_mean = np.mean(group_embeddings, axis=0)
        
        # Calculate distances from group mean
        distances = np.linalg.norm(group_embeddings - group_mean, axis=1)
        
        # Get indices of 8 closest questions (or fewer if group is smaller)
        top_8_local_indices = np.argsort(distances)[:8]
        
        # Map local indices back to original indices
        top_8_global_indices = group_indices[top_8_local_indices]
        
        # Store in dictionary
        top_questions_per_group[str(group)] = top_8_global_indices.tolist()
    
    return top_questions_per_group

# Usage would look like:
result = select_top_questions_per_group(group_numbers, questions, embeddings)
print(result)

{'0': [4504, 3003, 3722, 3575, 3623, 4255, 3938, 3436], '1': [4603, 4750, 7094, 3351, 2511, 7188, 4846, 2917], '2': [3874, 4370, 4515, 4377, 4188, 4024, 4369, 4489], '3': [1937, 1765, 2317, 6467, 3797, 2264, 2485, 3952], '4': [3906, 3791, 3778, 3753, 3808, 1851, 4027, 2299], '5': [2748, 2593, 3552, 2909, 3267, 3149, 2785, 3480], '6': [7412, 6843, 3549, 3301, 7436, 7168, 7227, 3025], '7': [6468, 6710, 6165, 6046, 5867, 5821, 6322, 5770], '8': [256, 5, 1175, 1669, 794, 6124, 398, 6556], '9': [7293, 5417, 7045, 7199, 6953, 6838, 6851, 7421], '10': [6458, 6620, 141, 6116, 6550, 6587, 6229, 5815], '11': [2238, 2343, 2048, 1835, 2181, 1775, 1768, 2404], '12': [4642, 5427, 5261, 4825, 5305, 5181, 5348, 5150], '13': [4519, 2904, 3533, 3085, 2807, 2880, 2869, 3275], '14': [1592, 1369, 1641, 642, 892, 220, 199, 554], '15': [6208, 605, 2948, 2547, 947, 139, 1409, 2756], '16': [3692, 3401, 3662, 3390, 3316, 4015, 4483, 4542], '17': [403, 6395, 5658, 5960, 1583, 69, 1318, 1054], '18': [4220, 1575, 

/tmp/ipykernel_34958/4203012270.py:6: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  embeddings = np.array(embeddings.cpu())


In [8]:
def construct_grouped_dataset(data, result):
    # Initialize a dictionary to store grouped data
    grouped_dataset = {}
    
    # Iterate through the result dictionary
    for group, indices in result.items():
        # Select items from data using the indices
        grouped_dataset[group] = [data[idx] for idx in indices]
    
    return grouped_dataset

# Usage would look like:
grouped_dataset = construct_grouped_dataset(data, result)
print(grouped_dataset.keys())
print(len(grouped_dataset['0']))  # number of items in group 0

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19'])
8


In [11]:
# with open(os.path.join(dataset_root, 'train_gr20_samples.json'), 'w') as f:
#     json.dump(grouped_dataset, f, indent=2)

if len(group_numbers) == len(data):
    for i, item in enumerate(data):
        for key, val in item.items():
            val['group_number'] = group_numbers[i]

In [13]:
data[0]['0'].keys()

dict_keys(['conversations', 'system', 'tools', 'group_number'])

In [14]:
with open(os.path.join(dataset_root, 'train_gr20.json'), 'w') as f:
    json.dump(data, f, indent=2)